# MAXCUT Landscape Analysis

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, Aer, execute
from qiskit.algorithms.optimizers import COBYLA, ADAM
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.utils import QuantumInstance
from qiskit_optimization.applications import Maxcut
from qiskit.opflow import AerPauliExpectation, PauliSumOp
from qiskit.quantum_info import Statevector
from qiskit.circuit import Parameter

import numpy as np
from qiskit import Aer
from qiskit.quantum_info import Statevector
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import itertools


In [2]:
G = nx.random_regular_graph(3, 4)
# Generate the adjacency matrix
adjacency_matrix = nx.adjacency_matrix(G)
max_cut = Maxcut(adjacency_matrix)
qubitOp, offset = max_cut.to_quadratic_program().to_ising()

/var/folders/pv/xwqtt1w56s7dkzm8bn7k9gx00000gn/T/ipykernel_52164/2781843548.py:3: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency_matrix = nx.adjacency_matrix(G)


In [9]:
def compute_expectation_value(params, qubitOp, qaoa):
    """ Compute expectation value for a set of gamma and beta values.
    params: tuple of gamma and beta values
    qubitOp: qubit operator
    qaoa: qaoa instance

    Returns: expectation value or None if an error occurs
    """
    gamma_vals, beta_vals = params
    try:
        # Construct the QAOA circuit
        qc = qaoa.construct_circuit(gamma_vals + beta_vals, operator=qubitOp)[0]
        backend = Aer.get_backend('aer_simulator')

        # Simulate the statevector and compute the expectation value
        statevector = Statevector.from_instruction(qc)
        expectation = statevector.expectation_value(qubitOp).real
        return expectation
    except Exception as e:
        # Log the error
        logging.error(f"Error in compute_expectation_value with params {params}: {e}")
        return None

def parallel_computation(gamma, beta, qubitOp, qaoa, p):
    """ Parallel computation of the objective function values.
    gamma: array of gamma values
    beta: array of beta values
    qubitOp: qubit operator
    qaoa: qaoa instance
    p: number of QAOA layers

    Returns: Dictionary of objective function values with parameter indices as keys
    """
    # Dictionary to store the results
    obj_vals = {}

    with ProcessPoolExecutor() as executor:
        futures = {}
        for gamma_vals in itertools.product(*[gamma] * p):
            for beta_vals in itertools.product(*[beta] * p):
                future = executor.submit(compute_expectation_value, (gamma_vals, beta_vals), qubitOp, qaoa)
                futures[future] = (gamma_vals, beta_vals)

        for future in tqdm(as_completed(futures), total=len(futures)):
            gamma_vals, beta_vals = futures[future]
            result = future.result()
            # Compute indices and use them as keys in the results dictionary
            gamma_indices = tuple(np.searchsorted(gamma, val) for val in gamma_vals)
            beta_indices = tuple(np.searchsorted(beta, val) for val in beta_vals)
            obj_vals[(gamma_indices, beta_indices)] = result

    return obj_vals


In [10]:
# Example usage for p > 1
p = 2  # Example for p > 1
qaoa = QAOA(optimizer=COBYLA(), reps=p)
num_points = 20

# Generate parameter grids
gamma = np.linspace(-2 * np.pi, 2 * np.pi, num_points)
beta = np.linspace(-2 * np.pi, 2 * np.pi, num_points)

# Run parallel computation
obj_vals = parallel_computation(gamma, beta, qubitOp, qaoa, p)


Process SpawnProcess-49:
Traceback (most recent call last):
  File "/Users/vivekkatial/.pyenv/versions/3.8.10/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/vivekkatial/.pyenv/versions/3.8.10/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/vivekkatial/.pyenv/versions/3.8.10/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/vivekkatial/.pyenv/versions/3.8.10/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'compute_expectation_value' on <module '__main__' (built-in)>
Process SpawnProcess-50:
Traceback (most recent call last):
  File "/Users/vivekkatial/.pyenv/versions/3.8.10/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/vivekkatial/.pyenv/versions/3.8.

  0%|                                                                                                                                                            | 0/160000 [00:00<?, ?it/s]


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table